<a href="https://colab.research.google.com/github/EHN8829/EJERCICIOS6A/blob/main/streamlit_ef.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
from googleapiclient.discovery import build
import pandas as pd
from textblob import TextBlob

# Título de la aplicación
st.title('Análisis de Sentimientos en YouTube')

# Entrada para la clave de API de YouTube
api_key = st.text_input('Introduce tu clave de API de YouTube')

# Entrada para la palabra clave de búsqueda
keyword = st.text_input('Introduce una palabra clave para buscar videos')

if api_key and keyword:
    # Construcción del cliente de la API de YouTube
    youtube = build("youtube", "v3", developerKey=api_key)

    # Realización de la búsqueda de videos usando la palabra clave
    request = youtube.search().list(
        part="snippet",
        q=keyword,
        type="video",
        maxResults=5  # Se puede ajustar el número de resultados
    )

    response = request.execute()

    # Presentación de los títulos y los ID de los videos
    video_ids = []
    for item in response.get('items', []):
        video_id = item["id"]["videoId"]
        video_ids.append(video_id)
        st.write(f'Título: {item["snippet"]["title"]}, Video ID: {video_id}')

    # Si hay videos, procedimiento para extraer comentarios y realizar el análisis de sentimientos
    if video_ids:
        all_comments = []

        for video_id in video_ids:
            # Obtener comentarios del video
            comments_request = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=50  # Limitar a 50 comentarios por video
            )

            comments_response = comments_request.execute()

            for comment_item in comments_response.get("items", []):
                comment = comment_item["snippet"]["topLevelComment"]["snippet"]["textOriginal"]
                all_comments.append(comment)

        # Presentación de los comentarios
        st.subheader('Comentarios extraídos')
        st.write(all_comments)

        # Análisis de sentimientos con TextBlob
        st.subheader('Análisis de Sentimientos')
        sentiment_scores = []

        for comment in all_comments:
            blob = TextBlob(comment)
            sentiment = blob.sentiment.polarity  # Adquisición polaridad (-1 a 1)
            sentiment_scores.append(sentiment)

        # Presentación de los resultados del análisis
        avg_sentiment = sum(sentiment_scores) / len(sentiment_scores) if sentiment_scores else 0
        st.write(f'El sentimiento promedio es: {avg_sentiment:.2f}')
        if avg_sentiment > 0:
            st.write('Sentimiento positivo en general')
        elif avg_sentiment < 0:
            st.write('Sentimiento negativo en general')
        else:
            st.write('Sentimiento neutral en general')